In [3]:
import requests
from bs4 import BeautifulSoup
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from urllib.parse import urlparse
import re
import time
from pprint import pprint 
from tqdm import tqdm
from typing import Optional, Callable

e:\Shamim\assignments\denv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from config import (
    ELASTICSEARCH_URL,
    INDEX_NAME_HUGGINGFACE,
    INDEX_NAME_GEMINI,
    INDEX_NAME_OPENAI,
    SITEMAP_URL,
    HUGGINGFACE_EMBEDDING_MODEL_NAME,
    GEMINI_EMBEDDING_MODEL_NAME,
    OPENAI_EMBEDDING_MODEL_NAME
)

**Connect To Elastic Search**

In [5]:
es = Elasticsearch(ELASTICSEARCH_URL)
print("Connected to Elasticsearch!")

client_info = es.info()
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'T1HeaWnRTOqX_BBgREVVbA',
 'name': '64c49e436740',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-10-21T10:06:21.288851013Z',
             'build_flavor': 'default',
             'build_hash': '25d88452371273dd27356c98598287b669a03eae',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.1',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.2.0'}}


**Query List**
# 
Since the doc is not public, I could not write script to read it. That's why I am using static list. In the future, we will 
use script to read data from a url.

In [30]:
query_list = [
    "What is baby botox and how does it work?",
    "Find sculptra providers in my area.",
    "What are fat dissolving injections for the chin?",
    "Where can I find botox treatments locally?",
    "What is lemon bottle fat dissolving?",
    "What is the cost of botox for neck pain treatment?",
    "lip flip services in Cork",
    "What are polynucleotides and how do they function?",
    "Provide details on Sculptra treatments available in Ireland.",
    "Where to get botox in Dublin",
    "Find Sculptra providers in my area.",
    "Find clinics offering Sculptra treatments in Ireland.",
    "Describe what Sculptra is and how it works.",
    "What is the cost of lip fillers in Ireland?",
    "What are effective neck tightening treatments?",
    "What is the cost of a thread lift in Ireland?",
    "Where can I find botox deals in Dublin?",
    "What are the benefits of eyebrow botox?",
    "What are the benefits of chin filler?",
    "Where can I get botox in Cork?",
    "Where can I get botox treatments in Waterford?",
    "What exactly is Sculptra?",
    "Explain Nefertiti Botox.",
    "How much does jawline filler cost",
    "What is facial cork used for",
    "What is blepharoplasty?",
    "Lip filler services in Galway",
    "How much does Sculptra cost in Dublin?",
    "Show me lip flip before and after results.",
    "Where can I find botox for 3 areas at 150?",
    "What are the best Sculptra clinics in Ireland?",
    "Find a blepharoplasty surgeon in Cork",
    "What are the best methods for under chin fat removal?",
    "Find beauty salons in Greystones.",
    "What are the benefits of filler under eyes?",
    "What is a profilo facial treatment",
    "What are the options for non-surgical rhinoplasty in Ireland?",
    "What is a Nefertiti neck lift procedure?",
    "What are the benefits of lip filler",
    "What are effective body fat reduction treatments?",
    "Where can I get botox in dublin",
    "Find clinics offering Sculptra in Cork.",
    "What are polynucleotides for under eyes",
    "Find Botox clinics in Dublin.",
    "Are there any botox offers available",
    "What is gummy smile botox treatment",
    "I need to find information about botox services in Dublin.",
    "lip flip procedure in cork",
    "Where to find botox services in Dublin City Centre?",
    "What is Sculptra?"
]

len(query_list)

50

Crate Index utils function

In [6]:
def create_index(index_name: str):
    if es.indices.exists(index=index_name):
        print(f"Index '{index_name}' already exists.")
        return

    mapping = {
        "mappings": {
            "properties": {
                "url": {"type": "keyword"},
                "title": {"type": "text"},
                "content": {"type": "text"},
                "embedding": {"type": "dense_vector", "dims": 384}
            }
        }
    }

    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created successfully.")

Extract page content utils function

In [7]:
def extract_page_text(url):
    try:
        res = requests.get(url, timeout=10)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "html.parser")

        # Remove unwanted elements
        for tag in soup(["script", "style", "noscript"]):
            tag.extract()

        title = soup.title.string.strip() if soup.title else "Untitled"
        text = re.sub(r"\s+", " ", soup.get_text(separator=" ", strip=True))
        return title, text

    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None, None

create_knn_query utils func

In [35]:
# The search query structure for k-NN
def create_knn_query(query_vector, k_value=5):
    """
    Creates an Elasticsearch k-NN search body.
    query_vector: The vector embedding of the user's query text.
    k_value: The number of nearest neighbors to return.
    """
    return {
        "knn": {
            "field": "embedding",  # Replace with your vector field name
            "query_vector": query_vector,
            "k": k_value,
            "num_candidates": 100  # Number of candidates to consider (tuning parameter)
        },
        # You may also include "fields" to limit the data returned for each hit
        "fields": ["url", "_id"]
    }

get_embedder model utils function

In [8]:
def get_embedder(model_type: str, api_key: Optional[str] = None) -> Callable[[str], list[float]]:
    """
    Initializes the embedding model and returns a function to encode text.
    
    Args:
        model_type: 'HuggingFace', 'Gemini', or 'OpenAI'.
        api_key: The necessary API key for the chosen service (if applicable).
        
    Returns:
        A callable function that takes a text string and returns a list of floats (embedding).
    """
    model_type = model_type.lower()

    if model_type == "huggingface":
        # Requires: pip install sentence-transformers
        # The specific model (e.g., 'all-MiniLM-L6-v2') can be passed as an argument
        try:
            from sentence_transformers import SentenceTransformer
        except ImportError:
            raise ImportError("HuggingFace embedder requires 'sentence-transformers'. Please install it.")
        
        # Initialize the model (e.g., a common sentence-transformer model)
        hf_model = SentenceTransformer(HUGGINGFACE_EMBEDDING_MODEL_NAME) 
        # Return a lambda that calls the encode method and converts to list
        return lambda content: hf_model.encode(content).tolist()

    elif model_type == "gemini":
        # Requires: pip install google-genai
        try:
            from google import genai
        except ImportError:
            raise ImportError("Gemini embedder requires 'google-genai'. Please install it.")
        
        if not api_key:
             raise ValueError("Gemini embedder requires an API key.")

        client = genai.Client(api_key=api_key)
        
        def gemini_embed(content: str) -> list[float]:
            # Use the 'embedding-001' model for general embeddings
            response = client.models.embed_content(
                model=GEMINI_EMBEDDING_MODEL_NAME,
                content=content,
                task_type='RETRIEVAL_DOCUMENT'
            )
            return response['embedding']

        return gemini_embed

    elif model_type == "openai":
        # Requires: pip install openai
        try:
            from openai import OpenAI
        except ImportError:
            raise ImportError("OpenAI embedder requires 'openai'. Please install it.")
        
        if not api_key:
             raise ValueError("OpenAI embedder requires an API key.")
             
        client = OpenAI(api_key=api_key)

        def openai_embed(content: str) -> list[float]:
            # Use the standard text-embedding-3-small model
            response = client.embeddings.create(
                input=content,
                model=OPENAI_EMBEDDING_MODEL_NAME
            )
            # The API returns a list of embeddings, we take the first (and only) one
            return response.data[0].embedding

        return openai_embed

    else:
        raise ValueError(f"Unknown model type: {model_type}. Must be 'HuggingFace', 'Gemini', or 'OpenAI'.")

scrape sitemap and index pages utils func

In [ ]:
def scrape_and_index(model_type: str, index_name: str, api_key: Optional[str] = None):
    '''
    Args:
        model_type: 'HuggingFace', 'Gemini', or 'OpenAI'.
        index_name: str
        api_key: The necessary API key for the chosen service (if applicable).
    Return: None
    '''
    # 1. Get Embedder Function
    try:
        # Get the callable function that performs the embedding
        embed_content = get_embedder(model_type, api_key=api_key) 
    except (ImportError, ValueError) as e:
        print(f"FATAL ERROR: Could not initialize embedder. {e}")
        return

    # 2. Scrape Sitemap
    print(f"Fetching sitemap from: {SITEMAP_URL}")
    sitemap_xml = requests.get(SITEMAP_URL).text
    soup = BeautifulSoup(sitemap_xml, "xml")

    # Filter URLs to the specific path
    urls = [loc.text for loc in soup.find_all("loc") if "/en-ie/" in loc.text]
    
    print(f"Found **{len(urls)}** URLs to crawl.")
    pprint(urls[:5])
    print("...")

    # 3. Process and Index Pages
    total_urls = len(urls)
    operations = []
    count_operation = 0
    batch_size = 50
    for url in tqdm(urls, total=total_urls, desc="Fetching and Indexing pages..."):
        count_operation += 1
        operations.append({"index": {"_index": index_name}})
        try:
            # Scrape content
            title, content = extract_page_text(url)
            
            # Skip if content is empty
            if not content:
                print(f"Skipped: Empty content for {url}")
                continue

            # Generate Embedding
            # Now we use the returned callable function 'embed_content'
            full_text = str(title) + str(content)
            embedding = embed_content(full_text)

            # Create Document
            doc = {
                "url": url,
                "title": title,
                "content": content,
                "embedding": embedding.tolist()
            }

            operations.append(doc)

            if count_operation >= batch_size:
                es.bulk(operations=operations)
                count_operation = 0
                operations.clear()

            # Index Document
            es.index(index=index_name, document=doc)
            # print(f"Indexed: {url}")

        except Exception as e:
            print(f"Failed to process/index {url}. Error: {e}")
    
    if operations:
        es.bulk(operations=operations)


**Scrape the pages from the sitemap and use HuggingFace Embedding Model**

In [19]:
create_index(index_name=INDEX_NAME_HUGGINGFACE)
scrape_and_index(model_type='HuggingFace', index_name=INDEX_NAME_HUGGINGFACE)

Index 'huggingface' created successfully.


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: e1d14493-30ed-4c55-8e51-b827a3f7db86)')' thrown while requesting HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/tokenizer_config.json
Retrying in 1s [Retry 1/5].


Fetching sitemap from: https://www.sisuclinic.com/sitemap.xml
Found **141** URLs to crawl.
['https://www.sisuclinic.com/en-ie/treatments',
 'https://www.sisuclinic.com/en-ie/providers',
 'https://www.sisuclinic.com/en-ie/faqs',
 'https://www.sisuclinic.com/en-ie/locations',
 'https://www.sisuclinic.com/en-ie/about-us']
...


Fetching and Indexing pages...:   1%|          | 1/141 [00:02<05:17,  2.27s/it]

Indexed: https://www.sisuclinic.com/en-ie/treatments


...

Fetching and Indexing pages...:  99%|█████████▉| 140/141 [02:47<00:00,  1.02it/s]

Failed to process/index https://www.sisuclinic.com/en-ie/blogs/why-men-get-anti-wrinkle-treatment. Error: BadRequestError(400, 'illegal_argument_exception', 'Malformed action/metadata line [57], expected field [create], [delete], [index] or [update] but found [url]')


Fetching and Indexing pages...: 100%|██████████| 141/141 [02:48<00:00,  1.19s/it]

Failed to process/index https://www.sisuclinic.com/en-ie/aesthetic-treatments/exosomes. Error: BadRequestError(400, 'illegal_argument_exception', 'Malformed action/metadata line [57], expected field [create], [delete], [index] or [update] but found [url]')


BadRequestError: BadRequestError(400, 'illegal_argument_exception', 'Malformed action/metadata line [57], expected field [create], [delete], [index] or [update] but found [url]')

In [20]:
count = es.count(index=INDEX_NAME_HUGGINGFACE)
count["count"]

148

In [31]:
len(query_list)

50

In [36]:
embedding_model = get_embedder(model_type="HuggingFace")
embedded_query = embedding_model("What is baby botox and how does it work?")
query_body = create_knn_query(query_vector=embedded_query, k_value=5)

In [37]:
response = es.search(
    index=INDEX_NAME_HUGGINGFACE,
    body=query_body
)

response

ObjectApiResponse({'took': 1413, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 5, 'relation': 'eq'}, 'max_score': 0.76685464, 'hits': [{'_index': 'huggingface', '_id': 'TggxfpoBAdziKzBU8AY2', '_score': 0.76685464, '_source': {'url': 'https://www.sisuclinic.com/en-ie/our-clinics/botox-dysport-miami', 'title': 'Botoxâ\x84¢:Dysportâ\x84¢ Miami - Sisu Clinic', 'content': "Botoxâ\x84¢:Dysportâ\x84¢ Miami - Sisu Clinic Black Friday is live! Save up to 30% on select treatments. Try our AI Makeover now and preview your refreshed look for free. Try Now Ireland United States Ireland United Kingdom Treatments Patient Results Get The Look Locations Meet Your Provider Pricing Blogs Shop Book Today choose region Republic Of Ireland United States of America United Kingdom learn more View Anti-Wrinkle View Get The Look Offers View All Treatments View Pricing View All Locations View Patients Results Meet Your Provider Have a questio

In [42]:
hits = response["hits"]["hits"]
for hit in hits:
    print(hit["_score"])

0.76685464
0.76685464
0.7471477
0.7471477
0.7464458
